# NeRF: Neural Radiance Field

## Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np

from ipynb.config import InferConfig, TrainConfig
from ipynb.infer import Inferer
from ipynb.train import Trainer

%matplotlib inline

## Train

In [2]:
trainer = Trainer(TrainConfig(), verbose=True)
trainer.display()

In [3]:
if H:= trainer.history:
    plt.figure()
    plt.title("MSE")
    plt.plot([mse for mse, _ in H.train], label="train")
    plt.show()

    plt.figure()
    plt.title("PSNR")
    plt.plot([psnr for _, psnr in H.train], label="train")
    plt.show()

## Inference

In [4]:
inferer = Inferer(InferConfig(), verbose=True)
inferer.display()

[BlenderDataset(hotdog, train)] Loading Data:   0%|          | 0/5 [00:00<?, ?it/s]

[BlenderDataset(hotdog, train)] Building Rays:   0%|          | 0/5 [00:00<?, ?it/s]

[Setup] Dataset Ready
[Setup] Model Ready
[Setup] Raymarcher Ready
[Setup] Creating Rays


[BlenderDataset(hotdog, train)] Building Rays:   0%|          | 0/120 [00:00<?, ?it/s]

[Setup] Rendering Started


...

[NeRF] Rendering:   0%|          | 0/2 [00:00<?, ?it/s]

[NeRF] Rendering:   0%|          | 0/2 [00:00<?, ?it/s]

[NeRF] Rendering:   0%|          | 0/2 [00:00<?, ?it/s]

[NeRF] Rendering:   0%|          | 0/2 [00:00<?, ?it/s]

[Setup] Rendering Done 
MoviePy - Building file ./res/NeRF_hotdog_pred.gif with imageio.
